# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA
* Model: DistilBERT
* Evaluation approach: evaluate method with a Hugging Face Trainer
* Fine-tuning dataset: dair-ai/emotion

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
! pip install -q "datasets==2.15.0"

In [2]:
from datasets import load_dataset

dataset = load_dataset("dair-ai/emotion", split="train").train_test_split(
    test_size=0.2, shuffle=True, seed=42
)

splits = ["train", "test"]

# View the dataset characteristics
dataset["train"]

Dataset({
    features: ['text', 'label'],
    num_rows: 12800
})

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [4]:
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["text"], truncation=True), batched=True
    )

# Inspect the available columns in the dataset
tokenized_dataset["train"]

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 12800
})

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=6,
    id2label={0: "sadness", 1: "joy", 2: "love", 3: "anger", 4: "fear", 5: "surprise"},
    label2id={"sadness": 0, "joy": 1, "love": 2, "anger": 3, "fear": 4, "surprise": 5},
)


for param in model.base_model.parameters():
    param.requires_grad = False

model.classifier

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Linear(in_features=768, out_features=6, bias=True)

In [6]:
'''if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))'''

"if tokenizer.pad_token is None:\n    tokenizer.add_special_tokens({'pad_token': '[PAD]'})\n    model.resize_token_embeddings(len(tokenizer))"

In [7]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./sentiment_analysis_original",
        learning_rate=2e-3,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=5,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)

trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.281700,1.251047,0.541250
2,1.253000,1.196605,0.542188
3,1.182500,1.095921,0.583438
4,1.138200,1.087668,0.596250
5,1.101800,1.075724,0.597812


Checkpoint destination directory ./sentiment_analysis_original/checkpoint-3200 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./sentiment_analysis_original/checkpoint-6400 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./sentiment_analysis_original/checkpoint-9600 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./sentiment_analysis_original/checkpoint-12800 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./sentiment_analysis_original/checkpoint-16000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=16000, training_loss=1.2081627082824706, metrics={'train_runtime': 251.1048, 'train_samples_per_second': 254.874, 'train_steps_per_second': 63.718, 'total_flos': 577865046509712.0, 'train_loss': 1.2081627082824706, 'epoch': 5.0})

In [8]:
trainer.evaluate()

{'eval_loss': 1.0757235288619995,
 'eval_accuracy': 0.5978125,
 'eval_runtime': 7.263,
 'eval_samples_per_second': 440.591,
 'eval_steps_per_second': 110.148,
 'epoch': 5.0}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [9]:
from peft import LoraConfig, TaskType
config = LoraConfig(target_modules=["q_lin", "k_lin", "v_lin"],
                    task_type=TaskType.SEQ_CLS)

In [10]:
for param in model.parameters():
    param.requires_grad = True
    
from peft import get_peft_model
lora_model = get_peft_model(model, config)

In [11]:
lora_model.print_trainable_parameters()

trainable params: 1,411,596 || all params: 67,774,476 || trainable%: 2.082784085265373


In [12]:
lora_trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./sentiment_analysis_lora",
        learning_rate=2e-3,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=5,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"].rename_column('label', 'labels'),
    eval_dataset=tokenized_dataset["test"].rename_column('label', 'labels'),
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)

lora_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.821900,0.909187,0.803750
2,0.805300,0.998076,0.744687
3,0.752200,0.861380,0.819063
4,0.649100,0.802454,0.827187
5,0.498700,0.623072,0.847187


Checkpoint destination directory ./sentiment_analysis_lora/checkpoint-3200 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./sentiment_analysis_lora/checkpoint-6400 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./sentiment_analysis_lora/checkpoint-9600 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./sentiment_analysis_lora/checkpoint-12800 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./sentiment_analysis_lora/checkpoint-16000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=16000, training_loss=0.760426959991455, metrics={'train_runtime': 633.6464, 'train_samples_per_second': 101.003, 'train_steps_per_second': 25.251, 'total_flos': 588804744352032.0, 'train_loss': 0.760426959991455, 'epoch': 5.0})

In [13]:
lora_model.save_pretrained("distilBERT-lora")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [14]:
from peft import AutoPeftModelForSequenceClassification

lora_model = AutoPeftModelForSequenceClassification.from_pretrained("distilBERT-lora",
                                                                    num_labels=6,
                                                                    output_attentions=False,
                                                                    output_hidden_states=False,
                                                                    ignore_mismatched_sizes=True)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
import pandas as pd

items_for_manual_review = tokenized_dataset["test"].select(
    [0, 1, 22, 31, 43, 292, 448, 487]
)

results = trainer.predict(items_for_manual_review)
lora_results = lora_trainer.predict(items_for_manual_review)
df = pd.DataFrame(
    {
        "text": [item["text"] for item in items_for_manual_review],
        "predictions": results.predictions.argmax(axis=1),
        "lora_predictions": lora_results.predictions.argmax(axis=1),
        "labels": results.label_ids
    }
)
# Show all the cell
pd.set_option("display.max_colwidth", None)
df

,text,predictions,lora_predictions,labels
0,while cycling in the country,1,1,4
1,i had pocket qq and was feeling pretty confident lol,1,1,1
2,i feel discouraged and beaten down i do better when i can pray about it obviously and then call my mama and friends,0,0,0
3,i could loose my job i would be so f amp ed for xmas i hate xmas i hate holidays i wish they would go away i feel nervous i feel sad what if i disappoint my family my friends,4,4,4
4,i feel scared when my father suddenly opens a door,4,4,4
5,i think it makes me feel like the heir to an incredibly rich and diverse legacy of stories and experiences,1,1,1
6,i could feel it but it didnt hurt,0,0,0
7,i haul of each to the lava planet and export them down to the space port feeling fairly clever,1,1,1


In [78]:
from transformers import TextClassificationPipeline
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

lora_model.to(device)
pipeline_lora = TextClassificationPipeline(model=lora_model, tokenizer=tokenizer, return_all_scores=True)

text = "This is unacceptable... I will not tolerate this!" #this should be classified as label 3 --> angry

print(pipeline_lora([text]))

The model 'PeftModelForSequenceClassification' is not supported for . Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GPT2ForSequenceClassification', 'GPT2ForSequenceCl

[[{'label': 'LABEL_0', 'score': 0.13361577689647675}, {'label': 'LABEL_1', 'score': 0.015379492193460464}, {'label': 'LABEL_2', 'score': 0.0030588689260184765}, {'label': 'LABEL_3', 'score': 0.8383708000183105}, {'label': 'LABEL_4', 'score': 0.009488294832408428}, {'label': 'LABEL_5', 'score': 8.685170178068802e-05}]]


In [79]:
model.to("cpu")
pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)

print(pipeline([text]))

[[{'label': 'sadness', 'score': 0.13361530005931854}, {'label': 'joy', 'score': 0.015379476360976696}, {'label': 'love', 'score': 0.0030588635709136724}, {'label': 'anger', 'score': 0.8383712768554688}, {'label': 'fear', 'score': 0.009488255716860294}, {'label': 'surprise', 'score': 8.685167995281518e-05}]]
